In [ ]:
import json
import pandas as pd
tickers = {}
with open('dow_jones_ticker_id.json', 'r') as ticker_file:
    data = json.load(ticker_file)
    for d in data:
        ticker_id = d['data'][0]['id']
        ticker_name = d['data'][0]['attributes']['name']
        tickers = tickers | {ticker_id: ticker_name}

In [ ]:
with open('repaired_all_transcripts.json', 'r') as f:
    data = json.load(f)


In [ ]:


contents = []
for d in data:
    #print(f"ticker id is {d['relationships']['primaryTickers']['data'][0]['id']}, content is {json.loads(d['content'])[0]['content']}")
    contents.append({"ticker_id":d['relationships']['primaryTickers']['data'][0]['id'], 
                     'content':json.loads(d['content'])[0]['content'],
                     'ts':d['attributes']['publishOn'],})
    #break

In [ ]:
pd_contents = pd.DataFrame(contents)

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#model.config.output_attentions = True  # 返回注意力权重
model.config.output_hidden_states = True  # 返回隐藏状态（可选）

In [9]:

import torch
script_hidden_states = []
for i in range(len(pd_contents)):
      print(f"i is {i} content len is {len(pd_contents.iloc[i]['content'])}")

      with torch.no_grad():
            output = model(input_ids=tokenizer(pd_contents.iloc[i]['content'], return_tensors="pt").input_ids.to("cuda"),
                  output_hidden_states=True,
                  return_dict=True,
                  use_cache=False
            )
      pooled_hidden_state = [o.mean(dim=1) for o in output.hidden_states[1:]] # (batch_size, sequence_length, hidden_size)
      script_hidden_state = torch.cat(pooled_hidden_state, dim=0)
      script_hidden_states.append(script_hidden_state.to('cpu').to(torch.float32))
      
      # 显式释放 GPU 内存
      del output, pooled_hidden_state, script_hidden_state
      torch.cuda.empty_cache()  # 清理缓存
      #pd_contents.loc[pd_contents.index[0], 'hidden_states'] = script_hidden_states.cpu().numpy().tolist()
      #break
      # if i > 200:
      #       break

i is 0 content len is 46084
i is 1 content len is 46391
i is 2 content len is 46429
i is 3 content len is 47240
i is 4 content len is 45124
i is 5 content len is 49567
i is 6 content len is 47085
i is 7 content len is 46836
i is 8 content len is 45809
i is 9 content len is 47247
i is 10 content len is 46681
i is 11 content len is 47470
i is 12 content len is 45012
i is 13 content len is 49049
i is 14 content len is 49091
i is 15 content len is 47966
i is 16 content len is 47994
i is 17 content len is 49362
i is 18 content len is 45913
i is 19 content len is 46483
i is 20 content len is 46016
i is 21 content len is 48476
i is 22 content len is 47022
i is 23 content len is 44919
i is 24 content len is 46533
i is 25 content len is 47538
i is 26 content len is 47026
i is 27 content len is 46092
i is 28 content len is 45518
i is 29 content len is 46349
i is 30 content len is 45918
i is 31 content len is 45630
i is 32 content len is 45723
i is 33 content len is 49597
i is 34 content len is 4

In [ ]:
pd_contents.loc[0]

In [10]:

import h5py

output_file = "hidden_states.h5"
with h5py.File(output_file, "w") as f:
    for i in range(len(pd_contents)):
        content = pd_contents.loc[i]
        id = content.ticker_id
        ticker_name = tickers[id]
        if ticker_name in f:
            g = f[ticker_name]
        else:
            g = f.create_group(tickers[id])
        #g.attrs["ts"] = ts.strftime("%Y-%m-%d %H:%M:%S")
        ts = content.ts
        g.create_dataset(f"hidden_states_{ts}", data=script_hidden_states[i])
        
        # if i > 200:
        #     break
        
    #f.create_dataset("hidden_states", data=script_hidden_states.cpu().numpy())
    #f.create_dataset("hidden_states", data=script_hidden_states.cpu().numpy(), compression="gzip")

In [41]:
# 函数：向嵌套字典添加新值
def add_to_nested_dict(nested_dict, k1, k2, value):
    # 如果 k1 不存在，创建新的子字典
    if k1 not in nested_dict:
        nested_dict[k1] = {}
    
    # 如果 k2 不存在，创建新的列表
    if k2 not in nested_dict[k1]:
        nested_dict[k1][k2] = []
    
    # 将 value 追加到 k2 对应的列表
    nested_dict[k1][k2].append(value)

In [45]:
from datetime import datetime
all_content = {}
with h5py.File(output_file, "r") as f_read:
    for k in f_read.keys():
        #print("文件中的键：", list(f_read[k]))
        for ds in list(f_read[k]):

            ts = datetime.strptime(ds.split("_")[-1], "%Y-%m-%d %H:%M:%S%z")
            day = str(ts.date())
            #print(day)
            # {k: {ts: f_read[k][ds][:]}}
            # all_content[k] = {ts.date():f_read[k][ds][:]}
            add_to_nested_dict(all_content, k, day, f_read[k][ds][:])
        #data = f_read[k]['hidden_states_2005-10-13 19:00:40+00:00'][:]
        #break

In [2]:
all_content.keys()

NameError: name 'all_content' is not defined

In [1]:
def swap_hierarchy(A):
    """
    交换字典 A 的层级，从 {name: {timestamps: [value, value]}} 转换为
    {timestamps: {name: [value, value]}}.
    
    参数:
        A: 输入字典，结构为 {name: {timestamps: [value, value]}}
    
    返回:
        new_A: 转换后的字典，结构为 {timestamps: {name: [value, value]}}
    """
    new_A = {}
    
    # 遍历原始字典
    for name, timestamp_dict in A.items():
        for timestamp, values in timestamp_dict.items():
            # 如果 timestamp 不在 new_A 中，初始化
            if timestamp not in new_A:
                new_A[timestamp] = {}
            # 将 name 和 values 添加到对应的 timestamp 下
            new_A[timestamp][name] = values
    
    return new_A

# 示例字典
A = {
    "name1": {"2023-01-01": [10, 20], "2023-06-01": [30, 40]},
    "name2": {"2023-01-01": [5, 15], "2023-07-01": [25, 35]}
}

# 转换层级
result = swap_hierarchy(A)
print(result)

{'2023-01-01': {'name1': [10, 20], 'name2': [5, 15]}, '2023-06-01': {'name1': [30, 40]}, '2023-07-01': {'name2': [25, 35]}}


In [21]:
torch.tensor(data)

tensor([[ 3.4180e-02,  1.1292e-02,  3.9551e-02,  ..., -2.6978e-02,
          2.1973e-02, -1.5869e-02],
        [ 2.7588e-02,  2.7924e-03,  5.8350e-02,  ..., -2.4567e-03,
          2.5269e-02, -5.0537e-02],
        [ 3.6133e-02, -5.0659e-03,  6.8848e-02,  ...,  2.8442e-02,
          3.5400e-02, -4.1748e-02],
        ...,
        [-5.2490e-02, -1.5723e-01, -4.0625e-01,  ...,  4.7461e-01,
          4.9561e-02,  6.5918e-02],
        [-1.0889e-01,  3.3447e-02, -7.6660e-02,  ...,  1.0703e+00,
          3.5352e-01, -1.0703e+00],
        [-1.8359e+00, -3.3906e+00, -2.1094e+00,  ...,  1.3125e+00,
         -1.0078e+00,  2.0625e+00]])

In [ ]:
with pd.HDFStore("hidden_states_pandas.h5") as store:
    store["seq_001"] = pd.Series(tmp.flatten())  # 需要展平或调整格式

In [ ]:
store.keys()

In [ ]:
pd_contents.loc[pd_contents.index[0], 'hidden_states'] = 'sdfsf'

In [ ]:
pd_contents

In [ ]:
model

In [ ]:
pooled_hidden_states = [o for o in output.hidden_states[1:]] # (batch_size, sequence_length, hidden_size)
script_hidden_states = torch.cat(pooled_hidden_states, dim=0)

In [ ]:
script_hidden_states.shape

In [ ]:
print(json.loads(data[4]['content'])[0]['content'])